In [1]:
%load_ext kedro.ipython

[07/17/24 00:26:18] INFO     Registered line magic '%reload_kedro'                                   ]8;id=879364;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=720927;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=957708;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=865049;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#60\60]8;;\

                    INFO     Resolved project path as: /Users/wilsvenleong/Downloads/Learning       ]8;id=312142;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=747565;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             Materials/healthhub-content-optimization/content-optimization.                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[07/17/24 00:26:19] INFO     Kedro project Content Optimization                                     ]8;id=610518;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=275954;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=145215;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=590792;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

[07/17/24 00:26:20] INFO     Registered line magic 'run_viz'                                        ]8;id=840622;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=115266;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [ ]:
import numpy as np
import pandas as pd
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance  # KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP

In [2]:
# ruff: noqa: F821
catalog.list()


[
    'all_contents',
    'all_contents_standardized',
    'all_contents_extracted',
    'all_extracted_text',
    'merged_data',
    'raw_word_counts',
    'log_word_counts',
    'flag_for_removal_by_type',
    'filtered_data_with_keywords',
    'parameters',
    'params:cfg',
    'params:cfg.option',
    'params:cfg.contributor',
    'params:cfg.to_remove',
    'params:selection_options',
    'params:selection_options.only_confirmed',
    'params:selection_options.all',
    'params:keywords',
    'params:keywords.model',
    'params:keywords.spacy_pipeline',
    'params:keywords.stop_words',
    'params:keywords.workers',
    'params:keywords.use_mmr',
    'params:keywords.diversity',
    'params:keywords.top_n',
    'params:columns_to_keep',
    'params:columns_to_keep.cost-and-financing',
    'params:columns_to_keep.diseases-and-conditions',
    'params:columns_to_keep.health-statistics',
    'params:columns_to_keep.live-healthy-articles',
    'params:columns_to_keep.medical-care-

In [3]:
# ruff: noqa: F821
filtered_data_with_keywords = catalog.load("filtered_data_with_keywords")
filtered_data_with_keywords.head()

                    INFO     Loading data from filtered_data_with_keywords (ParquetDataset)...  ]8;id=385262;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=650242;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body,keywords_all-MiniLM-L6-v2
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"<div class=""ExternalClass07C58E0D957B4AA7B14FC...",...,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...,"[cost, preventive measures, singapore citizens..."
1,1437742,Preventing HPV infection: HPV Vaccination,Preventing HPV infection: HPV Vaccination,"Sexual Health and Relationships,Child and Teen...",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/f...,faqs-on-hpv-and-hpv-immunisation,The Human Papillomavirus (HPV) vaccine or HPV ...,"<div class=""ExternalClass50E49E9021B24103B8719...",...,None,True,True,[],"[[[Cervarix (HPV2 vaccine), Gardasil 9 (HPV9 v...","[[screened, /a-z/costs-and-financing/cervical-...","[[About HPV and its link to Cervical Cancer, h...",[],About HPV and its link to Cervical Cancer\n\nW...,"[common surfaces, genital warts, cervical intr..."
2,1437732,Fever in Children,Fever in Children,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/c...,common-childhood-conditions-fever,Spot the symptoms of fever in your child and l...,"<div class=""ExternalClassA0AA5FCB88FC42E4AEF14...",...,None,False,False,"[Give Your Child the Best Protection, Fever]",None,"[[Upper Respiratory Tract Infections, https://...","[[What is a Fever in a Child?, h2], [How to Me...",[],"What is a Fever in a Child?\nA normal, healthy...","[upper respiratory tract, common viral infecti..."
3,1437433,"Hand, Foot, and Mouth Disease","Hand, Foot, and Mouth Disease",None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/hfmd,hfmd,"Hand, Foot, and Mouth Disease is present all y...","<div class=""ExternalClassB1300026922048DE8306D...",...,None,False,True,"[Hand, Foot, and Mouth Disease, Skin Disorders...",None,"[[Hand, Foot and Mouth Disease, https://www.he...","[[Signs and Symptoms, h2], [Complications, h2]...","[Symptoms of Hand, Food and Mouth Disease]","Hand, Foot, and Mouth Disease (HFMD) is caused...","[ev71 virus, childs temperature, wash hands, s..."
4,1437894,Cervical Cancer,Cervical Cancer,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/t...,topic_cervical_cancer,Cervical cancer is the 10th most common cancer...,"<div class=""ExternalClass4A8C8F03738343228D53E...",...,None,False,True,"[HPV Vaccination FAQ, Getting Support (for STI...",None,"[[HPV Vaccination FAQ, https://www.healthhub.s...","[[Link Between Cervical Cancer and HPV, h2], [...","[Pap Smear Illustration, Mother and Daughter, ...",The best protection against cervical cancer is...,"[daughter, immunosuppression, best protection,..."


In [4]:
content_category = "live-healthy-articles"
df = filtered_data_with_keywords.query(
    "content_category == @content_category"
).reset_index(drop=True)
df

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body,keywords_all-MiniLM-L6-v2
0,1444475,"Weight, BMI and Health Problems","Weight, BMI and Health Problems","Food and Nutrition,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/weight_p...,www.healthhub.sg/live-healthy/weight_putting_m...,weight_putting_me_at_risk_of_health_problems,What’s your Body Mass Index (BMI)? Learn how t...,"<div class=""ExternalClassE93BEC3784C545A286BB8...",...,None,False,False,"[BMI Calculator, What is a Healthy Weight?, An...",None,"[[BMI Calculator, https://www.healthhub.sg/pro...","[[What's a Healthy Body Mass Index?, h2], [Why...",[],What's a Healthy Body Mass Index?\nWe have all...,"[index, high blood pressure, important nutrien..."
1,1445137,7-month-baby Diet: An Authoritative Guide by O...,7-month-baby Diet: An Authoritative Guide by O...,"Food and Nutrition,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/meal-ide...,www.healthhub.sg/live-healthy/meal-ideas-month-7,meal-ideas-month-7,Your little one is now 7 months of age. Should...,"<div class=""ExternalClass46E64333542C4D8CBEA23...",...,None,True,True,"[Nutrition for Your Toddler, No Wholegrain, No...","[[[Meat and others, Milk (breast/formula)], [1...","[[Nutrition for Your Toddler, https://www.heal...","[[Recommended Number of Servings (7 months), h...","[7-month baby diet includes pureed veggies, Av...",By Health Promotion Board in collaboration wit...,"[different speeds, 60ml breast milk, brown ric..."
2,1445282,Older Adults Need More Protein,Older Adults Need More Protein,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/seniors-...,www.healthhub.sg/live-healthy/seniors-need-mor...,seniors-need-more-protein,Did you know that one in two older adults do n...,"<div class=""ExternalClass6C0CAB7C67934853B19AD...",...,None,True,True,[],"[[[Age Range, Protein required (in grams), Rec...","[[processed meats could increase your risk, ht...","[[How Much Protein do you Need?, h2], [Key Sou...",[Bread with peanut butter and eggs provides a ...,"As you age, your body requires roughly 50% mor...","[seniors, sarcopenia, muscle mass, diet concer..."
3,1443111,How Much Calories Do I Need A Day?,How Much Calories Do I Need A Day?,"Food and Nutrition,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/how-much...,www.healthhub.sg/live-healthy/how-much-to-eat-...,how-much-to-eat-at-each-meal,If you're not already aware of your daily calo...,"<div class=""ExternalClass2A6938FBCD87406FAF535...",...,None,False,True,"[Cut 100 Calories From Your Breakfast, Lunch a...",None,"[[Healthy365 app, https://www.healthhub.sg/pro...",[[Figuring out Your Daily Calorie Intake Isnt ...,[Choosing healthier options and eating fewer c...,Figuring out Your Daily Calorie Intake Isnt Ro...,"[female singaporean, healthier dining identifi..."
4,1443987,All You Need to Know About Childhood Immunisat...,All You Need to Know About Childhood Immunisat...,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/all-you-...,www.healthhub.sg/live-healthy/all-you-need-to-...,all-you-need-to-know-about-vaccinations,"To prevent diseases such as measles and mumps,...","<div class=""ExternalClass7601311C055C43B9A5813...",...,None,True,True,"[Information Is the Best Defence, How Healthy ...",[None],"[[National Childhood Immunisation Schedule, ht...","[[How Do Vaccines Work?, h2], [When Is It Not ...",[Getting more information about immunisation w...,By Associate Professor TAN Thiam Chye Head & S...,"[singapore, best defence, single injection, mm..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [6]:
# Extract the raw content body text
docs = df["extracted_content_body"].to_list()
len(docs)

634

In [7]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    low_memory=False,
    random_state=42,
)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(
    min_samples=5,
    min_cluster_size=15,
    metric="euclidean",  # manhattan
    cluster_selection_method="leaf",
    prediction_data=True,
)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 1))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Step 6 - (Optional) Fine-tune topic representations with
# a `bertopic.representation` model
# representation_model = KeyBERTInspired()
representation_model = MaximalMarginalRelevance(diversity=0.3)

# All steps together
topic_model = BERTopic(
    embedding_model=embedding_model,  # Step 1 - Extract embeddings
    umap_model=umap_model,  # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,  # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,  # Step 5 - Extract topic words
    representation_model=representation_model,  # Step 6 - (Optional) Fine-tune topic represenations
    # nr_topics=15,  # default is none, will auto reduce topics using HDBSCAN
)


topics, probs = topic_model.fit_transform(docs)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [8]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,77,-1_fat_diet_eat_calcium,"[fat, diet, eat, calcium, protein, healthier, ...",[You look at yourself in the mirror. Your once...
1,0,61,0_dementia_mental_stress_esteem,"[dementia, mental, stress, esteem, social, res...",[Has your child ever had a meltdown while stru...
2,1,51,1_healthier_food_meals_eat,"[healthier, food, meals, eat, rice, vegetables...","[A plate with grilled fish, brown rice, fruits..."
3,2,50,2_screen_myopia_activities_children,"[screen, myopia, activities, children, skills,...",[This article was written in collaboration wit...
4,3,45,3_influenza_infections_vaccines_doctor,"[influenza, infections, vaccines, doctor, infe...","[Fight the Flu Virus\nInfluenza and Hand, Foot..."
5,4,45,4_exercises_intensity_fitness_strength,"[exercises, intensity, fitness, strength, work...",[How to Manage Your Weight the Healthy Way\nRe...
6,5,39,5_pregnancy_delivery_babys_newborn,"[pregnancy, delivery, babys, newborn, bleeding...",[You are all excited that your baby is arrivin...
7,6,39,6_rice_sauce_chicken_garlic,"[rice, sauce, chicken, garlic, recipe, pan, wh...",[Everyday Solutions For Cooking Wholegrain Foo...
8,7,39,7_quit_nicotine_milestone_withdrawal,"[quit, nicotine, milestone, withdrawal, smoker...",[You know that cigarette smoking is harmful. Y...
9,8,32,8_diabetes_caffeine_risk_drinks,"[diabetes, caffeine, risk, drinks, cholesterol...",[The glycaemic index (or GI for short) uses a ...


In [9]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,What's a Healthy Body Mass Index?\nWe have all...,13,13_bmi_obesity_percentile_overweight,"[bmi, obesity, percentile, overweight, childre...",[By Health Promotion Board in collaboration wi...,bmi - obesity - percentile - overweight - chil...,0.674718,False
1,By Health Promotion Board in collaboration wit...,10,10_food_kids_nutrition_snacks,"[food, kids, nutrition, snacks, eating, vegeta...",[Nutrition for Your One-Year-Old\nYour little ...,food - kids - nutrition - snacks - eating - ve...,1.000000,False
2,"As you age, your body requires roughly 50% mor...",-1,-1_fat_diet_eat_calcium,"[fat, diet, eat, calcium, protein, healthier, ...",[You look at yourself in the mirror. Your once...,fat - diet - eat - calcium - protein - healthi...,0.000000,False
3,Figuring out Your Daily Calorie Intake Isnt Ro...,-1,-1_fat_diet_eat_calcium,"[fat, diet, eat, calcium, protein, healthier, ...",[You look at yourself in the mirror. Your once...,fat - diet - eat - calcium - protein - healthi...,0.000000,False
4,By Associate Professor TAN Thiam Chye Head & S...,3,3_influenza_infections_vaccines_doctor,"[influenza, infections, vaccines, doctor, infe...","[Fight the Flu Virus\nInfluenza and Hand, Foot...",influenza - infections - vaccines - doctor - i...,1.000000,False
...,...,...,...,...,...,...,...,...
629,Fat is not all bad...\nFat plays an important ...,-1,-1_fat_diet_eat_calcium,"[fat, diet, eat, calcium, protein, healthier, ...",[You look at yourself in the mirror. Your once...,fat - diet - eat - calcium - protein - healthi...,0.000000,False
630,A fragrant brown rice dish with raisins\n\nIng...,6,6_rice_sauce_chicken_garlic,"[rice, sauce, chicken, garlic, recipe, pan, wh...",[Everyday Solutions For Cooking Wholegrain Foo...,rice - sauce - chicken - garlic - recipe - pan...,1.000000,False
631,Meeting nutritional needs of kids and teenager...,10,10_food_kids_nutrition_snacks,"[food, kids, nutrition, snacks, eating, vegeta...",[Nutrition for Your One-Year-Old\nYour little ...,food - kids - nutrition - snacks - eating - ve...,0.788572,False
632,So you're pretty happy with the way you've bee...,0,0_dementia_mental_stress_esteem,"[dementia, mental, stress, esteem, social, res...",[Has your child ever had a meltdown while stru...,dementia - mental - stress - esteem - social -...,1.000000,False


In [10]:
similar_topics, similarity = topic_model.find_topics("diabetes", top_n=5)

In [11]:
similar_topics

[8, -1, 13, 15, 1]

In [12]:
topic_model.get_topic(similar_topics[0])


[
    ('diabetes', 0.5950085819748852),
    ('caffeine', 0.3047816501547118),
    ('risk', 0.30294351574287415),
    ('drinks', 0.302570586295863),
    ('cholesterol', 0.2739938179108366),
    ('teas', 0.2639579842623963),
    ('kopi', 0.25570959890239314),
    ('sugars', 0.2408292797045662),
    ('intake', 0.22672719326942936),
    ('factors', 0.2250239403160973)
]

In [13]:
doc_titles = df["title"].to_list()
urls = df["full_url"].to_list()

In [14]:
top_topics = topic_model.get_topic_freq()["Topic"].tolist()
reduced_embeddings = topic_model.umap_model.embedding_
hover_data = [
    f"{title} - {url} - Topic {topic}"
    for title, url, topic in zip(doc_titles, urls, topics)
]
visualization = topic_model.visualize_documents(
    hover_data,
    reduced_embeddings=reduced_embeddings,
    topics=top_topics,
    title="Topics",
)
visualization.show()

In [ ]:
visualization_barchart = topic_model.visualize_barchart(top_n_topics=len(top_topics))
visualization_barchart.show()

In [ ]:
visualization.write_html("visualization.html")
visualization_barchart.write_html("word_scores.html")

### Search for best parameters for HDBSCAN

- Get the parameters that gives the best DBCV scores
- Update the parmeters in Step 3 if needed
- Rerun BERTopic


In [ ]:
embeddings = topic_model.umap_model.embedding_  # reduced embeddings from umap

best_score = 0

for min_cluster_size in range(2, 11):
    for min_samples in [1, 2, 3, 4, 5, 6, 7]:
        for cluster_selection_method in ["eom", "leaf"]:
            for metric in ["euclidean", "manhattan"]:
                # for each combination of parameters of hdbscan
                hdb = HDBSCAN(
                    min_cluster_size=min_cluster_size,
                    min_samples=min_samples,
                    cluster_selection_method=cluster_selection_method,
                    metric=metric,
                    gen_min_span_tree=True,
                ).fit(embeddings)
                # DBCV score
                score = hdb.relative_validity_
                if score > best_score:
                    best_score = score
                    best_parameters = {
                        "min_cluster_size": min_cluster_size,
                        " min_samples": min_samples,
                        "cluster_selection_method": cluster_selection_method,
                        "metric": metric,
                    }

print(f"Best DBCV score: {best_score:.3f}")
print(f"Best parameters: {best_parameters}")

### Hierachical clustering of topics


In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
hierarchical_topics

In [ ]:
fig = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
fig.show()

In [ ]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

### Topics that are similar can be merged


In [ ]:
distance_matrix = cosine_similarity(np.array(topic_model.topic_embeddings_))
dist_df = pd.DataFrame(
    distance_matrix,
    columns=topic_model.topic_labels_.values(),
    index=topic_model.topic_labels_.values(),
)

tmp = []
for rec in dist_df.reset_index().to_dict("records"):
    t1 = rec["index"]
    for t2 in rec:
        if t2 == "index":
            continue
        tmp.append({"topic1": t1, "topic2": t2, "distance": rec[t2]})

pair_dist_df = pd.DataFrame(tmp)

pair_dist_df = pair_dist_df[
    (pair_dist_df.topic1.map(lambda x: not x.startswith("-1")))
    & (pair_dist_df.topic2.map(lambda x: not x.startswith("-1")))
]
pair_dist_df = pair_dist_df[pair_dist_df.topic1 < pair_dist_df.topic2]
pair_dist_df.sort_values("distance", ascending=False).head(
    10
)  # Top 10 most similar pairs

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(
    docs, calculate_tokens=True
)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])
df